# Building Text Similarity

# Initialization

In [1]:
!pip install wikipedia

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
import wikipedia

In [6]:
article_ids = [
    'Adam_Smith',
    'Bill_Clinton',
    'Bill_Gates',
    'Economics',
    'Free_market'
]

In [7]:
article_summaries = [wikipedia.summary(artId) for artId in article_ids]

In [8]:
tfidf = TfidfVectorizer().fit_transform(article_summaries[:2])
pairwise_similarity = tfidf * tfidf.T

In [11]:
pairwise_similarity.sum()

3.240747451406616

In [12]:
tfidf = TfidfVectorizer().fit_transform(article_summaries[-2:])
pairwise_similarity = tfidf * tfidf.T
pairwise_similarity.sum()

2.905346824130591

In [14]:
!ls sdow/

LICENSE          database         requirements.txt setup.cfg
README.md        docs             scripts          sketch
config           env              sdow             website


In [15]:
import pandas as pd

In [17]:
import file

ModuleNotFoundError: No module named 'file'

0

In [95]:
review_pattern = re.compile('.*review: (.*)$')
urls = review_pattern.findall(msg)[0]

a = ' '.join(arrLines[0])
re.match('.[0-9]+ (?! )+', a, 'r')

NameError: name 'msg' is not defined

In [74]:
a

' 1814 Adam_Smith "Adam Smith  (16 June [O.S. 5 June] 1723 – 17 July 1790) was a Scottish economist, philosopher and author as well as a moral philosopher, a pioneer of political economy and a key figure during the Scottish Enlightenment, also known as \'\'The Father of Economics\'\' or \'\'The Father of Capitalism\'\'. Smith wrote two classic works, The Theory of Moral Sentiments (1759) and An Inquiry into the Nature and Causes of the Wealth of Nations (1776). The latter, often abbreviated as The Wealth of Nations, is considered his magnum opus and the first modern work of economics. In his work, Adam Smith introduced his theory of absolute advantage.Smith studied social philosophy at the University of Glasgow and at Balliol College, Oxford, where he was one of the first students to benefit from scholarships set up by fellow Scot John Snell. After graduating, he delivered a successful series of public lectures at the University of Edinburgh, leading him to collaborate with David Hume 

In [53]:
f = open("sdow/database/subset_summaries.csv", "r")

arrLines = []
for line in f.readlines()[:20]:
#     print(line)
    arrLines.append(re.split("(^\d+) ", line))
    
f.close()

In [62]:
dfSummaries = pd.DataFrame(arrLines, columns=['_empty', 'nid', 'pageid'])
dfSummaries = dfSummaries[dfSummaries._empty != '\n'].drop('_empty', axis=1).reset_index(drop=True)
display(dfSummaries.shape)
dfSummaries

(10, 2)

,nid,pageid
0,1814,"Adam_Smith ""Adam Smith (16 June [O.S. 5 June]..."
1,3356,"Bill_Clinton ""William Jefferson Clinton (born ..."
2,3747,"Bill_Gates ""William Henry Gates III (born Oct..."
3,8182,"Dwight_D._Eisenhower ""Dwight David """"Ike"""" Eis..."
4,9223,"Economics ""Economics () is the social science ..."
5,10979,"Franklin_D._Roosevelt ""Franklin Delano Rooseve..."
6,11826,"Free_market ""In economics, a free market is a ..."
7,11955,"George_H._W._Bush ""George Herbert Walker Bush ..."
8,13765,"Henry_Kissinger ""Henry Alfred Kissinger (; Ger..."
9,14567,"International_trade ""International trade is th..."


In [92]:
matTfidf = TfidfVectorizer().fit_transform(dfSummaries.pageid)
pairwise_similarity = matTfidf * matTfidf.T


In [88]:
import numpy as np

In [ ]:
dfSummaries.style.background_gradient()

In [94]:
pd.DataFrame(pairwise_similarity.toarray()).style.background_gradient()

,0,1,2,3,4,5,6,7,8,9
0,1,0.413442,0.342691,0.454026,0.307186,0.443706,0.327578,0.351035,0.28984,0.181025
1,0.413442,1,0.399569,0.564972,0.245153,0.571397,0.309988,0.550832,0.382949,0.203786
2,0.342691,0.399569,1,0.434845,0.222044,0.426248,0.264118,0.338502,0.288769,0.164899
3,0.454026,0.564972,0.434845,1,0.24815,0.621431,0.318293,0.490276,0.408197,0.189376
4,0.307186,0.245153,0.222044,0.24815,1,0.28727,0.285446,0.207652,0.20018,0.197055
5,0.443706,0.571397,0.426248,0.621431,0.28727,1,0.342282,0.51586,0.422078,0.203397
6,0.327578,0.309988,0.264118,0.318293,0.285446,0.342282,1,0.266711,0.247503,0.21096
7,0.351035,0.550832,0.338502,0.490276,0.207652,0.51586,0.266711,1,0.332329,0.174205
8,0.28984,0.382949,0.288769,0.408197,0.20018,0.422078,0.247503,0.332329,1,0.183939
9,0.181025,0.203786,0.164899,0.189376,0.197055,0.203397,0.21096,0.174205,0.183939,1
